# 📚 Marching Cubes Exercise (12 Points)
**From Scalar Fields to Watertight Meshes**  
*Implementation with Parallel Processing*

## 🧩 1. Environment Setup
Import core dependencies and lookup tables:

In [1]:
import numpy as np
import multiprocessing as mp
from utils.consts import EDGE_TABLE, TRIANGLE_TABLE, EDGE_VERTICES

## ⚙️ 2. Core Class Structure (8 Points)
Initialize the Marching Cubes processor with parallel capabilities:
- **Exercise 1:** Complete⚡ 5. Vertex Interpolation using the following information

**Voxel Coordinates (i,j,k):**
   - The base coordinates of the current cube in the 3D grid
   - Cube spans from (i,j,k) to (i+1,j+1,k+1)

**Linear Interpolation (t):**
   - Calculates intersection point along edge using formula:
     `t = (iso - val1)/(val2 - val1)`
   - Only computed when edge crosses isosurface (one value ≥ iso, other < iso)

**Local Offsets (dx, dy, dz):**
   - Determined by corner index's binary representation
   - Example: Corner 3 (binary 011) → dx=1, dy=1, dz=0

**Global Position Calculation:**
   ```python
   x = i + dx1 + t*(dx2 - dx1)
   ```
   - `i` = voxel's x-coordinate
   - `dx1` = start corner's x-offset (0 or 1)
   - `dx2 - dx1` = direction vector component (always 0 except for the edge's axis)

In [ ]:
class MarchingCubes:
    def __init__(self, scalar_field, isovalue=0.0, num_processes=1):
        """
        Initialize Marching Cubes algorithm with scalar field and isovalue.

        Args:
            scalar_field (np.ndarray): 3D scalar field (density values)
            isovalue (float): Surface threshold value
            num_processes (int): Number of processes for parallelization
        """
        self.scalar_field = scalar_field
        self.isovalue = isovalue
        self.num_processes = num_processes

    def generate(self):
        """
        Generate mesh vertices and triangles using Marching Cubes with multiprocessing.

        Returns:
            vertices (np.ndarray): Unique vertex coordinates (N, 3)
            triangles (np.ndarray): Triangle indices (M, 3)
        """
        chunks = self._create_chunks()
        all_triangles = self._process_chunks(chunks)
        vertices, triangles = self._deduplicate_vertices(all_triangles)
        return vertices, triangles

    ## 🧊 3. Parallel Processing Setup
    # Chunking strategy for multi-core execution:
    def _create_chunks(self):
        """Split voxel indices along x-axis for parallel processing."""
        nx = self.scalar_field.shape[0]
        x_indices = np.arange(nx - 1)
        return np.array_split(x_indices, self.num_processes)

    def _process_chunks(self, chunks):
        """Process each chunk in parallel and collect triangles."""
        with mp.Pool(self.num_processes) as pool:
            args = [(self.scalar_field, self.isovalue, chunk) for chunk in chunks]
            results = pool.map(self._process_chunk, args)
        return [tri for chunk_tris in results for tri in chunk_tris]

    ## 🔬 4. Voxel Processing Core
    # Cube configuration analysis and edge detection:
    @staticmethod
    def _process_chunk(args):
        """Static method to process chunk of voxels (compatible with multiprocessing)."""
        scalar_field, isovalue, x_range = args
        nx, ny, nz = scalar_field.shape
        triangles = []

        for i in x_range:
            for j in range(ny - 1):
                for k in range(nz - 1):
                    # Get 8 corner values for current voxel
                    voxel_values = [
                        scalar_field[i, j, k], scalar_field[i+1, j, k],
                        scalar_field[i, j+1, k], scalar_field[i+1, j+1, k],
                        scalar_field[i, j, k+1], scalar_field[i+1, j, k+1],
                        scalar_field[i, j+1, k+1], scalar_field[i+1, j+1, k+1]
                    ]
                    # Calculate cube index
                    cube_index = sum((1 << idx) for idx, val in enumerate(voxel_values) if val >= isovalue)
                    # Get active edges and generate triangles
                    edge_table_entry = EDGE_TABLE[cube_index]
                    if edge_table_entry == 0:
                        continue
                    # Compute vertices for active edges
                    edge_verts = MarchingCubes._compute_edge_vertices(i, j, k, voxel_values, edge_table_entry, isovalue)
                    # Generate triangles using triangle table
                    triangles += MarchingCubes._generate_triangles(cube_index, edge_verts)
        return triangles

    ## ⚡ 5. Vertex Interpolation
    @staticmethod
    def _compute_edge_vertices(i, j, k, voxel_values, edge_table_entry, isovalue):
        """Compute interpolated vertices for active edges in a voxel using Marching Cubes.

        Args:
            i, j, k (int): Voxel grid coordinates (x, y, z) of the current cube's origin
            voxel_values (list[float]): 8 scalar values at cube corners (ordered 0-7 per MC convention)
            edge_table_entry (int): Bitmask from edge table indicating active edges (12-bit flag)
            isovalue (float): Surface threshold value for interpolation

        Returns:
            dict: Edge number to (x,y,z) coordinates of interpolated vertices
        """
        edge_verts = {}  # Stores {edge_number: (x,y,z)} for active edges with intersections

        # Iterate through all 12 potential cube edges
        for edge_num in range(12):
            # Skip if edge isn't active (bit not set in edge_table_entry)
            if not (edge_table_entry & (1 << edge_num)):
                continue

            # Get corner indices for this edge's endpoints (c1, c2)
            c1, c2 = EDGE_VERTICES[edge_num]

            # Get scalar values at both corners
            val1, val2 = voxel_values[c1], voxel_values[c2]

            # Skip if both corners are on same side of isosurface (no intersection)
            if (val1 >= isovalue) == (val2 >= isovalue):
                continue

            # Linear interpolation factor (0-1) between corners
            t = (isovalue - val1)/(val2 - val1)

            # Calculate local coordinates (within voxel) for both corners
            # Corner indices use 3-bit format: LSB = x, middle = y, MSB = z
            dx1, dy1, dz1 = (c1 & 1), ((c1 >> 1) & 1), ((c1 >> 2) & 1)
            dx2, dy2, dz2 = (c2 & 1), ((c2 >> 1) & 1), ((c2 >> 2) & 1)

            # Compute global coordinates using voxel origin (i,j,k) + interpolated offset
            # Only one axis changes between c1-c2 (dx2-dx1=1 for x-edge, etc.)
            # Your implementation should start here ...
            x = i + dx1 + t*(dx2 - dx1)
            y = j + dy1 + t*(dy2 - dy1)
            z = k + dz1 + t*(dz2 - dz1)

            edge_verts[edge_num] = (x, y, z)

        return edge_verts

    ## 🔺 6. Triangle Generation
    # Creating triangles from edge vertices:
    @staticmethod
    def _generate_triangles(cube_index, edge_verts):
        """Generate triangles for voxel using precomputed edge vertices."""
        triangles = []
        tri_table_entry = TRIANGLE_TABLE[cube_index]
        for i in range(0, len(tri_table_entry), 3):
            edges = tri_table_entry[i:i+3]
            if -1 in edges:
                break
            try:
                tri = [edge_verts[e] for e in edges]
                triangles.append(tri)
            except KeyError:
                continue  # Skip incomplete triangles (shouldn't occur with correct tables)
        return triangles

    ## 🧵 7. Vertex Deduplication
    # Creating unique vertex list:
    @staticmethod
    def _deduplicate_vertices(triangles):
        """Deduplicate vertices and remap triangle indices."""
        vertex_map = {}
        unique_vertices = []
        indices = []

        for tri in triangles:
            for vertex in tri:
                key = tuple(np.round(vertex, 6))
                if key not in vertex_map:
                    vertex_map[key] = len(unique_vertices)
                    unique_vertices.append(vertex)
                indices.append(vertex_map[key])

        # Reshape indices into triangles
        triangles = np.array(indices, dtype=np.int32).reshape(-1, 3)
        return np.array(unique_vertices), triangles

## 🧪 8. Test Surfaces (2 Points)
Implicit surface definitions:
- **Exercise 2: Complete the implicit function definition for Torus Surface**
- **Exercise 3: Complete the implicit function definition for Cube Surface**

In [3]:
# Test Cases
class ImplicitSurfaces:
    @staticmethod
    def sphere(resolution=(64, 64, 64), radius=1.0):
        """Generate scalar field for a sphere."""
        x = np.linspace(-2, 2, resolution[0])
        y = np.linspace(-2, 2, resolution[1])
        z = np.linspace(-2, 2, resolution[2])
        X, Y, Z = np.meshgrid(x, y, z, indexing='ij')
        return X**2 + Y**2 + Z**2 - radius**2

    @staticmethod
    def torus(resolution=(64, 64, 64), R=1.0, r=0.5):
        """
        Generate scalar field for a torus:
        The implicit equation for a torus centered at the origin is:

            (√(x² + y²) - R)² + z² = r²
            Where:

            R = Major radius (distance from center of tube to center of torus)

            r = Minor radius (radius of the tube)
        """
        # Your implementation should start here ...
        x = np.linspace(-2, 2, resolution[0])
        y = np.linspace(-2, 2, resolution[1])
        z = np.linspace(-2, 2, resolution[2])
        X, Y, Z = np.meshgrid(x, y, z, indexing='ij')
        return (np.sqrt(X**2 + Y**2) - R)**2 + Z**2 - r**2

    @staticmethod
    def cube(shape=(64, 64, 64), size=1.2):
        """
        Generate scalar field for a cube.
        The implicit equation for an axis-aligned cube centered at the origin
        is defined using the L-∞ norm (maximum norm):
            f(x,y,z) = max(|x|, |y|, |z|) - s
        Where:
            s = Half side length (distance from center to face)
            Surface exists where f(x,y,z) = 0
            Negative values → Inside cube
            Positive values → Outside cube
        """
        # Your implementation should start here ...
        x = np.linspace(-2, 2, shape[0])
        y = np.linspace(-2, 2, shape[1])
        z = np.linspace(-2, 2, shape[2])
        X, Y, Z = np.meshgrid(x, y, z, indexing='ij')
        return np.maximum(np.maximum(np.abs(X), np.abs(Y)), np.abs(Z)) - size

## 📊 9. Execution & Visualization (2 Points)
Test surface reconstruction:

In [4]:
vertices_list, triangles_list = [], []
if __name__ == "__main__":
    # Sphere test
    field = ImplicitSurfaces.sphere(resolution=(64, 64, 64))
    mc = MarchingCubes(field, num_processes=4)
    vertices, triangles = mc.generate()
    vertices_list.append(vertices)
    triangles_list.append(triangles)
    print(f"Sphere: {len(vertices)} vertices, {len(triangles)} triangles")

    # Torus test
    field = ImplicitSurfaces.torus(resolution=(64, 64, 64))
    mc = MarchingCubes(field, num_processes=4)
    vertices, triangles = mc.generate()
    vertices_list.append(vertices)
    triangles_list.append(triangles)
    print(f"Torus: {len(vertices)} vertices, {len(triangles)} triangles")

    # Cube test
    field = ImplicitSurfaces.cube(shape=(64, 64, 64))
    mc = MarchingCubes(field, num_processes=4)
    vertices, triangles = mc.generate()
    vertices_list.append(vertices)
    triangles_list.append(triangles)
    print(f"Cube: {len(vertices)} vertices, {len(triangles)} triangles")

Sphere: 4728 vertices, 9452 triangles
Torus: 7128 vertices, 14256 triangles
Cube: 8664 vertices, 17324 triangles


In [5]:
import open3d
for vertices, triangles in zip(vertices_list, triangles_list):
    mesh = open3d.geometry.TriangleMesh()
    # From numpy to Open3D vertices
    mesh.vertices = open3d.utility.Vector3dVector(vertices)
    # From numpy to Open3D triangles
    mesh.triangles = open3d.utility.Vector3iVector(triangles)
    open3d.visualization.draw_plotly(geometry_list=[mesh])

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


**Expected Results**:<br>
![Sphere](./../data/sphere_mc.png)
![Torus](./../data/torus_mc.png)
![Cube](./../data/cube_mc.png)